In [0]:
%pip install -U databricks-sdk==0.41.0 langchain-community==0.2.16 langchain-openai==0.1.19 mlflow==2.20.2
dbutils.library.restartPython()


## Compound AI system for your Instagram web campaign marketing for our Cookie Franchise!

<a href="https://youtu.be/UfbyzK488Hk?si=qzMgcSjcBhXOnDBz&t=3496" target="_blank"><img style="float:right; padding: 20px" width="500px" src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/llm-tools-functions/llm-tools-cookie-video.png?raw=true" ></a>

In this notebook, we're going to see how to create a compound AI system to analyze your data and make recommendation on what to publish in your Instagram web campaign, based on your own customer reviews!

we will:
- Create simple SQL functions to analyze your data and registering them in UC
- Creating more advanced python functions in UC
- Use Langchain to bind these functions as tools 
- Create an Agent that can execute these tools and ask it some tough questions


### AI with general intelligence isn't enough!

LLMs are very powerful, but they don't know your business. If we ask an LLM to write a post for our cookies, it'll generate a very general message.

AI needs be customized to your own business with your own data to be relevant!

Let's give it a try and ask a general AI to generate an instagram post for our cookie franchise:

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=1444828305810485&notebook=%2F01-agent-cookie-demo&demo_name=llm-tools-functions&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fllm-tools-functions%2F01-agent-cookie-demo&version=1&user_hash=032e6e2b1e1657acd615f0e41ec6018001a06a0376d5aa7a9cab91bc229e1e1d">

In [0]:
%sql
-- A not inspiring, general-purpose message :/ 
select ai_gen('Help me write an instagram message for the customers of my Seattle store, we want to increase our sales for the top 1 cookie only.')

## Generating personalized, impactful AI with the Data Intelligence Platform

Because Databricks makes it easy to bring all your data together, we can easily leverage your own data and use it to super-charge the AI.

Instead of General Intelligence, you'll have a extra-smart AI, understanding your business.

Let's see how Databricks let us generate ultra-personalize content and instagram messages to drive more cookie love!

### 1/ Prepare and review the dataset

Dbdemos loaded the dataset for you in your catalog. In a real world, you'd have a few Delta Live Table pipeline together with Lakeflow connect to ingest your ERP/Sales Force/website data.

Let's start by reviewing the dataset and understanding how it can help to make personalized recommendations.
*Note: the cookie data now exists in the marketplace, feel free to pull it from here.*

#### Let's focus on the main tables
- **sales.franchises** - Contains information about all our business franchises (store name, location, country, size, ect).   
- **sales.transactions** - Contains sales data for all franchises.
- **media.customer_reviews** - Social media reviews for a bunch of different franchises.   

In [0]:
%run ./_resources/00-init-cookie $reset_all=false

In [0]:
%sql
SELECT * FROM cookies_franchises;

In [0]:
%sql
SELECT * FROM cookies_transactions

## 2/ Create our Unity Catalog Functions

The cookie data now exists in your catalog.

We'll now start creating custom functions that our Compound AI system will understand and leverage to gain insight on your business

### Allow your LLM to retrieve the franchises for a given city

In [0]:
%sql
--Now we create our first function. This takes in a city name and returns a table of any franchises that are in that city.
--Note that we've added a comment to the input parameter to help guide the agent later on.
CREATE OR REPLACE FUNCTION
cookies_franchise_by_city ( city_name STRING COMMENT 'City to be searched' )
returns table(franchiseID BIGINT, name STRING, size STRING)
LANGUAGE SQL
-- Make sure to add a comment so that your AI understands what it does
COMMENT 'This function takes in a city name and returns a table of any franchises that are in that city.'
return
(SELECT franchiseID, name, size from cookies_franchises where lower(city)=lower(city_name) order by size desc)

In [0]:
%sql
-- Test the function we just created
SELECT * from cookies_franchise_by_city('Seattle')

### Sum all the sales for a given franchise, grouping the result per product

The AI will be able to find the top products generating the most revenue

In [0]:
%sql
--This function takes an ID as input, and this time does an aggregate to return the sales for that franchise_id.
CREATE OR REPLACE FUNCTION
cookies_franchise_sales (franchise_id BIGINT COMMENT 'ID of the franchise to be searched')
returns table(total_sales BIGINT, total_quantity BIGINT, product STRING)
LANGUAGE SQL
COMMENT 'This function takes an ID as input, and this time does an aggregate to return the sales for that franchise_id'
return
  (SELECT SUM(totalPrice) AS total_sales, SUM(quantity) AS total_quantity, product 
    FROM cookies_transactions 
      WHERE franchiseID = franchise_id GROUP BY product)

In [0]:
%sql
-- Test the function we just created
SELECT * from cookies_franchise_sales(3000005)

### Get the best feedbacks for a given franchise

This will let the AI understand what local customers like about our cookies!

In [0]:
%sql
--This function takes an ID as input, and this time does an aggregate to return the sales for that franchise_id.
CREATE OR REPLACE FUNCTION
cookies_summarize_best_sellers_feedback (franchise_id BIGINT COMMENT 'ID of the franchise to be searched')
returns STRING
LANGUAGE SQL
COMMENT 'This function will fetch the best feedback from a product and summarize them'
return
  SELECT AI_GEN(SUBSTRING('Extract the top 3 reason people like the cookies based on this list of review:' || ARRAY_JOIN(COLLECT_LIST(review), ' - '), 1, 80000)) AS all_reviews
  FROM cookies_customer_reviews
    where franchiseID = franchise_id and review_stars >= 4

In [0]:
%sql
-- Test the function we just created
SELECT cookies_summarize_best_sellers_feedback(3000005)

## 3/ Testing our AI with Databricks Playground

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/llm-tools-functions/llm-tools-functions-playground.gif?raw=true" width="500px" style="float: right; margin-left: 10px; margin-bottom: 10px;">


Your functions are now available in Unity Catalog!

To try out our functions with playground:
- Open the [Playground](/ml/playground) 
- Select a model supporting tools (like Llama3.1)
- Add the functions you want your model to leverage (`catalog.schema.function_name`)
- Ask a question (for example to write an instagram post for Seattle), and playground will do the magic for you!


### 4/ DIY: Create your own Agent chaining the tools with Langchain

In this step, we're going to define three cruicial parts of our agent:
- Tools for the Agent to use
- LLM to serve as the agent's "brains"
- System prompt that defines guidelines for the agent's tasks

In [0]:
from langchain_community.tools.databricks import UCFunctionToolkit
import pandas as pd
wh = get_shared_warehouse(name = None) #Get the first shared wh we can. See _resources/01-init for details
print(f'This demo will be using the {wh.name} to execute the functions')

def get_tools():
    return (
        UCFunctionToolkit(warehouse_id=wh.id)
        # Include functions as tools using their qualified names.
        # You can use "{catalog_name}.{schema_name}.*" to get all functions in a schema.
        .include(f"{catalog}.{db}.cookies_franchise_by_city", 
                 f"{catalog}.{db}.cookies_franchise_sales", 
                 f"{catalog}.{db}.cookies_summarize_best_sellers_feedback")
        .get_tools())

display_tools(get_tools()) #display in a table the tools - see _resource/00-init for details

In [0]:
from langchain_community.chat_models.databricks import ChatDatabricks

#We're going to use llama 3.3 because it's tool enabled and works great. Keep temp at 0 to make it more deterministic.
llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.0,
    streaming=False)

In [0]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatDatabricks

#This defines our agent's system prompt. Here we can tell it what we expect it to do and guide it on using specific functions. 

def get_prompt(history = [], prompt = None):
    if not prompt:
            prompt = """You are a helpful assistant for a global company that oversees cookie stores. Your task is to help store owners understand more about their products and sales metrics. You have the ability to execute functions as follows: 

            Use the franchise_by_city function to retrieve the franchiseID for a given city name.

            Use the franchise_sales function to retrieve the cookie sales for a given franchiseID.

           Use the cookies_summarize_best_sellers_feedback function to understand what customers like the most.

    Make sure to call the function for each step and provide a coherent response to the user. Don't mention tools to your users. Don't skip to the next step without ensuring the function was called and a result was retrieved. Only answer what the user is asking for. If a user ask to generate instagram posts, make sure you know what customers like the most to make the post relevant."""
    return ChatPromptTemplate.from_messages([
            ("system", prompt),
            ("human", "{messages}"),
            ("placeholder", "{agent_scratchpad}"),
    ])

In [0]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

prompt = get_prompt()
tools = get_tools()
agent = create_openai_tools_agent(llm, tools, prompt)

#Put the pieces together to create our Agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [0]:
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

#Very basic chain that allows us to pass the input (messages) into the Agent and collect the (output) as a string
agent_str = ({ "messages": itemgetter("messages")} | agent_executor | itemgetter("output") | StrOutputParser())

In [0]:
#Lets ask our Compound AI Agent to generate an Instagram post. This requires it to:
#     1. Look up what stores are in Chicago
#     2. Use sales data to look up the best selling cookie at that store
# COMING SOON:
#     3. Retrieve reviews from a vector search endpoint
#     4. Generate images from the shutterstock model

answer=agent_str.invoke({"messages": "Help me write an instagram message for the customers of my Seattle store, we want to increase our sales for the top 1 cookie only."})

### That's it! Our super customized review is ready:

In [0]:
print(answer)